# PDS A2
You will be shared ten files from coders who annotated verses from the 1st book of Iliad (no names shared). In each file there are the following columns: polarity (the sentiment the reader felt while reading the verse), emotions, hero (Homer narrating or a hero talking). The goal of this assignment is to build Machine Learning for automated sentiment annotation.  

---

### 1. Exploring the data.
  * Address the missing values (and any outliers). 
  * Measure inter-annotator agreement in the *polarity* and *hero* columns.

---

In [53]:
# T1
import os
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics import cohen_kappa_score as kappa
from itertools import combinations as com

In [54]:
warnings.filterwarnings("ignore") # Filter Warnings since it keeps giving warnings when dropping columns
path= r'C:..\IB1-annotated'
files = os.listdir(path) # Get the files from the path

In [55]:
filenames = [f for f in files if f[-4:] == 'xlsx'] # Choose only excel files. reference: https://stackoverflow.com/questions/56423421/read-all-xlsx-files-in-a-folder-and-save-the-files-in-different-dataframes answer by PirateNinjas
print(filenames)
os.chdir(path) # Change Directory so that it can read the excel files.
coders = pd.DataFrame() # Initiate the Data Frame
for f in filenames:
    df = pd.read_excel(f, sheet_name='annotation') #Make sure to read from the annotation sheet.
    df['ID'] = int(f[0:-5])
    coders = coders.append(df) # Create the Data Frame
coders

['3150014.xlsx', '3252107.xlsx', '3252108.xlsx', '3252113.xlsx', '335201.xlsx', '3352106.xlsx', '3352114.xlsx', '3352115.xlsx', '3352116.xlsx', '3352119.xlsx']


,verse,polarity,emotions,hero,ID,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,"Τη μάνητα, θεά, τραγούδα μας του ξακουστού Αχι...",positive,admiration,NaN,3150014,NaN,NaN,NaN,NaN
1,"ανάθεμα τη, πίκρες που 'δωκε στους Αχαιούς περ...",negative,pain,NaN,3150014,NaN,NaN,NaN,NaN
2,και πλήθος αντρειωμένες έστειλε ψυχές στον Άδη...,negative,awe,NaN,3150014,NaN,NaN,NaN,NaN
3,"παλικαριών, στους σκύλους ρίχνοντας να φάνε τα...",negative,fear,NaN,3150014,NaN,NaN,NaN,NaN
4,και στα όρνια ολούθε —έτσι το θέλησε να γίνει ...,no emotion,acknowledgement,NaN,3150014,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
584,κει που 'χε χτίσει στον καθένα τους παλάτι ο κ...,positive,joy,NaN,3352119,NaN,NaN,NaN,NaN
585,"ο ξακουστός τεχνίτης Ήφαιστος, με τη σοφή του ...",positive,respect,NaN,3352119,NaN,NaN,NaN,NaN
586,"Κι ο Δίας ο Ολύμπιος, ό αστραπόχαρος, στην κλί...",positive,respect,NaN,3352119,NaN,NaN,NaN,NaN
587,"εκεί που ως τώρα πάντα, ως του 'ρχονταν ύπνος ...",positive,joy,NaN,3352119,NaN,NaN,NaN,NaN


In [56]:
print(coders['Unnamed: 4'].unique(), coders['Unnamed: 5'].unique(), coders['Unnamed: 6'].unique(), coders['Unnamed: 7'].unique())

[nan 'CHRISIS' 'Kalhas' 'Hrisis' 'Muse' 'Calchas'] [nan] [nan] [nan]


In [57]:
coders.drop(['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7'], axis=1, inplace=True)
coders

,verse,polarity,emotions,hero,ID,Unnamed: 4
0,"Τη μάνητα, θεά, τραγούδα μας του ξακουστού Αχι...",positive,admiration,NaN,3150014,NaN
1,"ανάθεμα τη, πίκρες που 'δωκε στους Αχαιούς περ...",negative,pain,NaN,3150014,NaN
2,και πλήθος αντρειωμένες έστειλε ψυχές στον Άδη...,negative,awe,NaN,3150014,NaN
3,"παλικαριών, στους σκύλους ρίχνοντας να φάνε τα...",negative,fear,NaN,3150014,NaN
4,και στα όρνια ολούθε —έτσι το θέλησε να γίνει ...,no emotion,acknowledgement,NaN,3150014,NaN
...,...,...,...,...,...,...
584,κει που 'χε χτίσει στον καθένα τους παλάτι ο κ...,positive,joy,NaN,3352119,NaN
585,"ο ξακουστός τεχνίτης Ήφαιστος, με τη σοφή του ...",positive,respect,NaN,3352119,NaN
586,"Κι ο Δίας ο Ολύμπιος, ό αστραπόχαρος, στην κλί...",positive,respect,NaN,3352119,NaN
587,"εκεί που ως τώρα πάντα, ως του 'ρχονταν ύπνος ...",positive,joy,NaN,3352119,NaN


In [58]:
print(coders['hero'].unique())

[nan 'Chryses' 'Agamemnon' 'Achilleus' 'Kalhas' 'Athena' 'Nestor' 'Thetis'
 'Odysseus' 'Zeus' 'Hera' 'Hephaestus' 'Apollo' 'Patroclus' 'Homer'
 'Chrysis' 'Kalchas' 'Calchas' 'Menelaus' 'Andromache' 'Antenor' 'Phobos'
 'Iris' 'Briseis']


In [59]:
coders['hero'].isnull().sum()

1748

In [60]:
coders['Unnamed: 4'].notnull().sum()

120

In [61]:
#for i in range(len(coders)):
#if pd.isna(coders.hero[i]) and pd.notnull(coders['Unnamed: 4'][i]):  # reference: https://stackoverflow.com/questions/44877663/error-float-object-has-no-attribute-notnull answer by Max Kleiner
#coders['hero'][i] = coders['Unnamed: 4'][i]

coders.hero = np.where(coders.hero.isnull(), coders['Unnamed: 4'], coders.hero) # reference: https://stackoverflow.com/questions/30357276/how-to-pass-another-entire-column-as-argument-to-pandas-fillna answer by joris.
coders.drop(['Unnamed: 4'], axis=1, inplace=True)

In [62]:
coders['hero'].isnull().sum()

1628

In [63]:
coders['polarity'].isnull().sum()

199

In [64]:
coders.polarity.fillna('no emotions', inplace=True)
coders.polarity.isnull().sum()

0

In [65]:
coders.emotions.isnull().sum()

1297

In [66]:
coders.emotions.fillna('neutral', inplace=True)
coders.emotions.isnull().sum()

0

In [67]:
coders.hero.unique()

array([nan, 'Chryses', 'Agamemnon', 'Achilleus', 'Kalhas', 'Athena',
       'Nestor', 'Thetis', 'Odysseus', 'Zeus', 'Hera', 'Hephaestus',
       'Apollo', 'Patroclus', 'Homer', 'Chrysis', 'Kalchas', 'Calchas',
       'CHRISIS', 'Menelaus', 'Andromache', 'Hrisis', 'Antenor', 'Phobos',
       'Iris', 'Briseis', 'Muse'], dtype=object)

In [68]:
coders.hero.replace(['Chryses', 'CHRISIS', 'Hrisis', 'Chrisis', 'Kalchas', 'Kalhas'], ['Chryseis', 'Chrysis', 'Chrysis','Chrysis', 'Calchas', 'Calchas'], inplace=True)
coders.hero.fillna('Homer', inplace=True)

In [69]:
coders.hero.unique()

array(['Homer', 'Chryseis', 'Agamemnon', 'Achilleus', 'Calchas', 'Athena',
       'Nestor', 'Thetis', 'Odysseus', 'Zeus', 'Hera', 'Hephaestus',
       'Apollo', 'Patroclus', 'Chrysis', 'Menelaus', 'Andromache',
       'Antenor', 'Phobos', 'Iris', 'Briseis', 'Muse'], dtype=object)

In [70]:
coders.reset_index(inplace=True) # Reference on how to convert index into Column: https://datatofish.com/index-to-column-pandas-dataframe/

In [71]:
coders.rename(columns = {'index' : 'verse_id'}, inplace=True)
coders.set_index(['ID', 'verse_id'], inplace=True)

In [72]:
coders

verse  \
ID      verse_id                                                      
3150014 0         Τη μάνητα, θεά, τραγούδα μας του ξακουστού Αχι...   
        1         ανάθεμα τη, πίκρες που 'δωκε στους Αχαιούς περ...   
        2         και πλήθος αντρειωμένες έστειλε ψυχές στον Άδη...   
        3         παλικαριών, στους σκύλους ρίχνοντας να φάνε τα...   
        4         και στα όρνια ολούθε —έτσι το θέλησε να γίνει ...   
...                                                             ...   
3352119 584       κει που 'χε χτίσει στον καθένα τους παλάτι ο κ...   
        585       ο ξακουστός τεχνίτης Ήφαιστος, με τη σοφή του ...   
        586       Κι ο Δίας ο Ολύμπιος, ό αστραπόχαρος, στην κλί...   
        587       εκεί που ως τώρα πάντα, ως του 'ρχονταν ύπνος ...   
        588       κει πάνω ανέβη, κι η χρυσόθρονη σιμά του επλάγ...   

                    polarity         emotions   hero  
ID      verse_id                                      
3150014 0           positive       admiration  Homer  
        1           negative             pain  Homer  
        2           negative              awe  Homer  
        3           negative             fear  Homer  
        4         no emotion  acknowledgement  Homer  
...                      ...              ...    ...  
3352119 584         positive              joy  Homer  
        585         positive          respect  Homer  
        586         positive          respect  Homer  
        587         positive              joy  Homer  
        588         positive             hope  Homer  

[5890 rows x 4 columns]

In [73]:
coders.index.get_level_values(0).unique()

Int64Index([3150014, 3252107, 3252108, 3252113,  335201, 3352106, 3352114,
            3352115, 3352116, 3352119],
           dtype='int64', name='ID')

In [74]:
comb = list(com(coders.index.get_level_values(0).unique(), 2))
for i in range(len(comb)):
    print(comb[i][0], 'with' , comb[i][1])

3150014 with 3252107
3150014 with 3252108
3150014 with 3252113
3150014 with 335201
3150014 with 3352106
3150014 with 3352114
3150014 with 3352115
3150014 with 3352116
3150014 with 3352119
3252107 with 3252108
3252107 with 3252113
3252107 with 335201
3252107 with 3352106
3252107 with 3352114
3252107 with 3352115
3252107 with 3352116
3252107 with 3352119
3252108 with 3252113
3252108 with 335201
3252108 with 3352106
3252108 with 3352114
3252108 with 3352115
3252108 with 3352116
3252108 with 3352119
3252113 with 335201
3252113 with 3352106
3252113 with 3352114
3252113 with 3352115
3252113 with 3352116
3252113 with 3352119
335201 with 3352106
335201 with 3352114
335201 with 3352115
335201 with 3352116
335201 with 3352119
3352106 with 3352114
3352106 with 3352115
3352106 with 3352116
3352106 with 3352119
3352114 with 3352115
3352114 with 3352116
3352114 with 3352119
3352115 with 3352116
3352115 with 3352119
3352116 with 3352119


In [75]:
k1 = []
k2 = []
for i in range (0, len(comb)):
    k1.append(kappa(coders.loc[comb[i][0]].polarity, coders.loc[comb[i][1]].polarity))
    k2.append(kappa(coders.loc[comb[i][0]].hero, coders.loc[comb[i][1]].hero))
print(f'Polarity Cohen kappa: {np.mean(k1):.4f}\nHero Cohen kappa: {np.mean(k2):.4f}')

Polarity Cohen kappa: 0.1416
Hero Cohen kappa: 0.6925


### 2. Data preprocessing.
  * Map the `polarity` (positive, negative, no emotion) to scores (respectively: 1, -1, 0). 
  * Perform a data exploratory analysis for `polarity` by visualising the class balance per annotator, and the variance per verse and per annotator. Combine `polarity` with `emotions` and `hero` to explore the data further (for example, one could study the aggregated sentiment-score per hero/narrator or the emotion distribution across the sentiment scores). Note that all figures should comply with the ten rules of visualisation that were taught in class. 
  * Suggest three findings (max: 50 words each) that will be based on the visualisations of your exploratory analysis.

---

In [ ]:
# T2


### 3. Automated annotation.
 * Build baselines (at least one based on random guesses) and regressors (at least three sklearn-based) that will yield a score (from -1 to 1) estimating the reader's sentiment for an unseen verse. 
 * Evaluate your models using mean absolute error (MAE) and mean square error (MSE). Turn the gold and predicted scores to classes (-1, 0, 1) and evaluate also using *proper* classification evaluation metrics.  
 * Diagnose and analyse any under/over fitting. 
 * Announce a winner based on your evaluation and apply it in order to predict a label (not score) per verse on the 24th Iliad book that is given. Submit your predictions as a compressed CSV with the following title: `IB24.your-student-ID-number.csv.gz`, where `your-student-ID-number` will hold your student ID number). The submitted dataframe should comprise the verses in one column (exactly as the original), but it should also comprise another column to hold the aligned predictions. 

 ---

In [ ]:
# T3


### 4. Scraping and silver labeling.
 * Scrape all the books of Iliad, by using this [translation from Project Gutenberg](https://www.gutenberg.org/cache/epub/36248/pg36248-images.html).
 * Use your best performing sentiment classifier from (3) to label the verses of all the 24 crawled books (silver labeling).  
 * Visualise the sentiment series resulted from the silver labels of all the books. 
 * Evaluate your model's predictions for the 1st scraped book with the respective gold annotations (of the same book from T3, yet with a different translation), which you used to train your model.

In [ ]:
# T4

###  BONUS:  

Try to improve your best performing model, in order to better generalise. Your labels of the 24th will be compared against those of a quality annotator to evaluate your model. The top five scores will receive a bonus of 10%. 

---

In [3]:
# Bonus coding...


### Submission: 
* A zipped folder (only ZIP, not RAR), with your student ID number as name (e.g., f12345.zip), which will include the following files: 
  * The notebook with the tasks and the solutions, named as: `your-student-ID-number.ipynb`.
  * The dataframe with the ground truth annotations from the second task (T2) named as: `IB1.your-student-ID-number.csv.gz`
  * The predictions for the 24th book (from T3), named as: `IB24.your-student-ID-number.csv.gz`
  * The dataframe with the silver annotations of all the scrapped books (from T4), using a long format and named as: `iliad_from_gutenberg.your-student-ID-number.csv.gz`

### Evaluation criteria: 
  * The four tasks are equally weighted in terms of grades (25% each). 
  * With this assignment you are expected to do data preprocessing, exploratory analysis, train and evaluate machine learning models, employ (scraping and) visualisation as an analysis tool. 
  * The code cells that solve a task should follow the cell with the respective task description. Any textual analysis/description should exist in **text** cells (not in the source code) following the code cells that solve the related task. Use Jupyter's markdown-cell option to add text cells. 
  * If you borrow a solution that exists online, name the link you took it from and what you did to adapt it to your task. Detected plagiarism (esp. copying from a source without quoting and duplicate code between students) will lead to a zero grade.
  * Your code should be well-structured and comments should explain as much as possible, to avoid misunderstandings during evaluation (points might be lost due to this).
  * Everyone will be assessed by their written notebook, but if there are questions, some may be asked to explain in brief orally.

---